In [10]:
import feedparser
import requests

# Célula 2 (Nova - após os imports existentes)
from transformers import pipeline
from bs4 import BeautifulSoup

# Inicializa o pipeline de análise de sentimento com FinBERT
# Isso pode levar alguns minutos na primeira vez, pois o modelo será baixado.
try:
    sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert")
    print("Pipeline de análise de sentimento FinBERT carregado com sucesso.")
except Exception as e:
    print(f"Erro ao carregar o modelo FinBERT: {e}")
    sentiment_analyzer = None # Define como None para evitar erros posteriores se falhar


/Users/vitor/Projects/NewsSummaryV2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


Pipeline de análise de sentimento FinBERT carregado com sucesso.


In [12]:
outputs = []


def ler_feed_rss(url_feed):
    """
    Lê os itens de um feed RSS e retorna uma lista de notícias.\n

    Args:
        url_feed (str): A URL do feed RSS \n.
    Returns:,
        list: Uma lista de dicionários, onde cada dicionário representa uma notícia.\n,
        Retorna uma lista vazia em caso de erro.
    """
    print(f"Tentando ler o feed RSS de: {url_feed}\n")
    
    try:
        # Usar requests para buscar o feed com um user-agent, o que pode ajudar
        # a evitar bloqueios por alguns servidores.
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url_feed, headers=headers, timeout=10) # Timeout de 10 segundos
        response.raise_for_status() # Levanta um erro para respostas HTTP ruins (4xx ou 5xx)
        
        # Analisar o conteúdo do feed usando feedparser
        feed = feedparser.parse(response.content)

        if feed.bozo:
            print(f"Aviso: O feed pode estar malformado. Erro: {feed.bozo_exception}\n")

        print(f"Título do Feed: {feed.feed.get('title', 'N/A')}")
        print(f"Link do Feed: {feed.feed.get('link', 'N/A')}")
        print(f"Descrição do Feed: {feed.feed.get('description', 'N/A')}")
        print("--- Últimas Entradas ---")

        noticias_coletadas = []

        if not feed.entries:
            print("Nenhuma entrada encontrada no feed.")
            return noticias_coletadas

        for i, entrada in enumerate(feed.entries):
            
            titulo = entrada.get('title', 'Sem título')
            link = entrada.get('link', 'Sem link')
            publicado = entrada.get('published', 'Data não disponível')
            # Tenta obter um resumo ou a descrição completa
            
            resumo_html = entrada.get('summary', entrada.get('description', 'Sem resumo'))
            
            # Limpa tags HTML do resumo\n",
            if resumo_html:
                try:
                    # Assegure-se que BeautifulSoup foi importado no início do script/célula\n",
                    soup = BeautifulSoup(resumo_html, "html.parser")
                    resumo_limpo = soup.get_text(separator=' ', strip=True)
                except NameError: # Caso BeautifulSoup não esteja definido (ex: import falhou)
                    print("Aviso: BeautifulSoup não importado/disponível. Limpeza de HTML pode ser incompleta.")
                    import re
                    resumo_limpo = re.sub(r'<[^>]+>', '', resumo_html) # Limpeza básica com regex
                else:
                   resumo_limpo = " "
    
                noticias_coletadas.append({
                    'titulo': titulo,
                    'link': link,
                    'publicado': publicado,
                    'resumo': resumo_limpo
                })
            return noticias_coletadas
            # Você pode acessar outros campos também, se disponíveis:
            # print(f"  Autor: {entrada.get('author')}")
            # print(f"  Tags: {[tag.term for tag in entrada.get('tags', [])]}")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar o feed RSS: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
    return []


In [9]:

if __name__ == "__main__":
    # Exemplo de URL de feed RSS (substitua pela URL do feed que você quer ler)
    # Alguns exemplos de feeds RSS públicos:
    # url_g1 = "https://g1.globo.com/rss/g1/"
    # url_jovem_nerd = "https://jovemnerd.com.br/feed/"
    # url_stack_overflow_blog = "https://stackoverflow.blog/feed/"
    
    #url_do_feed_exemplo = "https://cms.zerohedge.com/fullrss2.xml" # Blog da Python Software Foundation
    # url_do_feed_exemplo = "https://www.investing.com/rss/news.rss" # Não tem resumo

    
    # url_do_feed_exemplo = "https://feeds.content.dowjones.io/public/rss/mw_topstories"
    url_do_feed_exemplo = "https://seekingalpha.com/feed.xml"


    if not url_do_feed_exemplo:
        print("Por favor, defina uma URL de feed RSS na variável 'url_do_feed_exemplo'.")
    else:
        ler_feed_rss(url_do_feed_exemplo)

    # Exemplo com um feed que pode não ter todos os campos
    # print("\n--- Testando outro feed ---")
    # ler_feed_rss("https://www.nasa.gov/rss/dyn/breaking_news.rss")


Tentando ler o feed RSS de: https://seekingalpha.com/feed.xml

Título do Feed: All Articles on Seeking Alpha
Link do Feed: https://seekingalpha.com
Descrição do Feed: © seekingalpha.com. Use of this feed is limited to personal, non-commercial use and is governed by Seeking Alpha's Terms of Use (https://about.seekingalpha.com/terms). Publishing this feed for public or commercial use and/or misrepresentation by a third party is prohibited.
--- Últimas Entradas ---

Entrada 1:
  Título: Achieve Life Sciences: FDA Approval Not Assured, But Valuation Justifies Risk
  Link: https://seekingalpha.com/article/4792147-achieve-life-sciences-fda-approval-not-assured-but-valuation-justifies-risk?source=feed_all_articles
  Publicado: Tue, 03 Jun 2025 18:28:07 -0400
  Resumo: Sem resumo...

Entrada 2:
  Título: SEB Still Facing Headwinds Amid More Economic Uncertainty
  Link: https://seekingalpha.com/article/4792146-skandinaviska-enskilda-banken-headwinds-amid-economic-uncertainty?source=feed_all_art